In [1]:
#creating spark instance
from pyspark.sql import SparkSession
spark = SparkSession .builder.appName('datamodeling').getOrCreate()

In [2]:
df = spark.read.option("sep","\t").csv("reducedusersrows.tsv",header='true', inferSchema='true')

In [3]:
from pyspark.sql.functions import monotonically_increasing_id

In [4]:
#indexing rows
newdataf=df.withColumn("rowId", monotonically_increasing_id())

In [5]:
#selecting the user ids having 1s
listids = [list(x.asDict().values())[0] for x in newdataf.filter(newdataf.post_event_list.rlike("^(1,[0-9 , . =]*|[0-9 , . =]*,1,[0-9 , . =]*|[0-9 , . =]*,1)$")).select("post_uniqueid").distinct().collect()]

In [6]:
count=newdataf.count()

In [7]:
#selecting all the userids
listidsall = [list(x.asDict().values())[0] for x in newdataf.select("post_uniqueid").collect()]

In [8]:
#if the post_event_list contains 1 for a particular userid, assign 1 as the label for the that userid throughout the dataframe 
label=[]
for i in range(0,count):
    if listidsall[i] in listids:
        label.append(1)
    else:
        label.append(0) 

In [9]:
#create a column for the list 'label'
import pandas as pd
labeling=pd.DataFrame(data=label,columns=['label'])

In [10]:
newdataP=newdataf.toPandas()

In [11]:
#join the column to the dataframe
result = newdataP.join(labeling)

In [27]:
df1 = result[['label','post_uniqueId']]

In [12]:
result.to_csv("with_label.tsv", sep="\t", index=False)